In [52]:

import sys
import pandas as pd 
import tensorflow as tf 
#import data_preparer

# df = pd.read_csv('D:/projects/ai/poc/homework/training_data_20210128.csv')
# df_test = pd.read_csv('D:/projects/ai/poc/homework/testing_data_20210128.csv')

# print(df.columns)
# print(test_df.columns)

In [53]:
df_train = pd.read_csv('D:/projects/ai/poc/homework/training_data_20210128.csv')
# df=df.head(1000)
#df = pd.read_excel(source_file_path)
#df = pd.read_csv(source_file_path)
df_train = df_train.loc[df_train['LOT_TYPE']=='FDY']
df_train = df_train.loc[df_train['LAYER']!='XX']
df_train['CHIPNAME'] = df_train['CHIPNAME'].str.strip()
df_train = df_train.loc[df_train['CHIPNAME']=='CHIP266']           

df_train['PRIORITY'] = df_train['PRIORITY'].astype(str)
df_train['IS_MAIN_ROUTE'] = df_train['IS_MAIN_ROUTE'].astype(str)
df_train['DATA_DATE'] = df_train['DATA_DATE'].astype(str)
          
df_train['DATA_DATE'] = pd.to_datetime(df_train['DATA_DATE'], infer_datetime_format=True)
df_train['WS_DATE'] = pd.to_datetime(df_train['WS_DATE'], infer_datetime_format=True)
df_train = df_train.assign(PROCESSED_DAYS = ((df_train['DATA_DATE'] -df_train['WS_DATE'])/pd.Timedelta(1, 'D')).fillna(0).astype('float64'))
df_train['ACTUAL_WP_OUT'] =pd.to_datetime(df_train['ACTUAL_WP_OUT'], infer_datetime_format=True)
df_train = df_train.assign(REMAIN_DAYS=((df_train['ACTUAL_WP_OUT']-df_train['DATA_DATE'])/pd.Timedelta(1, 'D')).fillna(0).astype('float64'))
df_train = df_train.assign(CYCLE_TIME=((df_train['ACTUAL_WP_OUT']-df_train['WS_DATE'])/pd.Timedelta(1, 'D')).fillna(0).astype('float64'))


In [54]:
def iqr(df, colname, bounds = [0, .75]):
    s = df[colname]
    q = s.quantile(bounds)
    return df[~s.clip(*q).isin(q)]


In [55]:
df_test = pd.read_csv('D:/projects/ai/poc/homework/testing_data_20210128.csv')
df_test = df_test.loc[df_test['LOT_TYPE']=='FDY']
df_test = df_test.loc[df_test['LAYER']!='XX']
df_test['CHIPNAME'] = df_test['CHIPNAME'].str.strip()
df_test = df_test.loc[df_test['CHIPNAME']=='CHIP266']   
     

df_test['PRIORITY'] = df_test['PRIORITY'].astype(str)
df_test['IS_MAIN_ROUTE'] = df_test['IS_MAIN_ROUTE'].astype(str)
df_test['DATA_DATE'] = df_test['DATA_DATE'].astype(str) 

df_test['DATA_DATE'] = pd.to_datetime(df_test['DATA_DATE'], infer_datetime_format=True)

df_test['WS_DATE'] = pd.to_datetime(df_test['WS_DATE'], infer_datetime_format=True)
df_test = df_test.assign(PROCESSED_DAYS = ((df_test['DATA_DATE'] -df_test['WS_DATE'])/pd.Timedelta(1, 'D')).fillna(0).astype('float64'))
df_test['ACTUAL_WP_OUT'] =pd.to_datetime(df_test['ACTUAL_WP_OUT'], infer_datetime_format=True)
df_test = df_test.assign(REMAIN_DAYS=((df_test['ACTUAL_WP_OUT']-df_test['DATA_DATE'])/pd.Timedelta(1, 'D')).fillna(0).astype('float64'))
df_test = df_test.assign(CYCLE_TIME=((df_test['ACTUAL_WP_OUT']-df_test['WS_DATE'])/pd.Timedelta(1, 'D')).fillna(0).astype('float64'))

In [56]:
# df_train =iqr(df_train,'CYCLE_TIME',[0, .75])     
# df_test =iqr(df_test,'CYCLE_TIME',[0, .75])        

In [57]:
df_train.columns

Index(['DATA_DATE', 'IDX', 'LOT_ID', 'STATUS', 'CHIPNAME', 'LAYER',
       'REMAIN_LAYER_SEQ', 'OP_NO', 'REMAIN_OP_SEQ', 'PRIORITY',
       ...
       'UM', 'UT', 'WL', 'WS', 'WT', 'ZL', 'ACTUAL_WP_OUT', 'PROCESSED_DAYS',
       'REMAIN_DAYS', 'CYCLE_TIME'],
      dtype='object', length=263)

In [58]:
 #df = df.drop(columns=['IDX','LOT_TYPE','WS_DATE','ACTUAL_WP_OUT','DATA_DATE','LAYER','LOT_ID'])
 df_train = df_train.drop(df_train.loc[:, '0E':'ZL'].columns, axis = 1) 

 #df_test = df_test.drop(columns=['IDX','LOT_TYPE','WS_DATE','ACTUAL_WP_OUT','DATA_DATE','LAYER','LOT_ID'])
 df_test = df_test.drop(df_test.loc[:, '0E':'ZL'].columns, axis = 1) 


print(df_test.columns)         

Index(['DATA_DATE', 'IDX', 'LOT_ID', 'STATUS', 'CHIPNAME', 'LAYER',
       'REMAIN_LAYER_SEQ', 'OP_NO', 'REMAIN_OP_SEQ', 'PRIORITY', 'LOT_TYPE',
       'WIP_QTY', 'WS_DATE', 'IS_MAIN_ROUTE', 'ACTUAL_WP_OUT',
       'PROCESSED_DAYS', 'REMAIN_DAYS', 'CYCLE_TIME'],
      dtype='object')


In [59]:
print(df_train.shape)
print(df_train.info())

print(df_test.shape)
print(df_test.info())

(14061, 18)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 14061 entries, 2242 to 222839
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATA_DATE         14061 non-null  datetime64[ns]
 1   IDX               14061 non-null  int64         
 2   LOT_ID            14061 non-null  object        
 3   STATUS            14061 non-null  object        
 4   CHIPNAME          14061 non-null  object        
 5   LAYER             14061 non-null  object        
 6   REMAIN_LAYER_SEQ  14061 non-null  float64       
 7   OP_NO             14061 non-null  object        
 8   REMAIN_OP_SEQ     14061 non-null  float64       
 9   PRIORITY          14061 non-null  object        
 10  LOT_TYPE          14061 non-null  object        
 11  WIP_QTY           14061 non-null  int64         
 12  WS_DATE           14061 non-null  datetime64[ns]
 13  IS_MAIN_ROUTE     14061 non-null  object        
 14  ACTUAL

In [60]:
#Categorical Columns like  
#Continuous Columns like  
#Requirement : I need count of output date

# print(pd.crosstab(train_df.out,columns='WIPDT',margins=True))
# print(pd.crosstab(test_df.out,columns='WIPDT',margins=True))

In [61]:
#Requirement: All Numerical columns basic statistics.
df_train.describe()

,IDX,REMAIN_LAYER_SEQ,REMAIN_OP_SEQ,WIP_QTY,PROCESSED_DAYS,REMAIN_DAYS,CYCLE_TIME
count,14061.000000,14061.000000,14061.000000,14061.000000,14061.000000,14061.000000,14061.000000
mean,244141.061304,13.032786,275.512055,24.907261,30.268925,30.432173,60.701098
std,145281.420496,8.351608,161.047537,0.957752,17.770886,17.861721,4.839966
min,8864.000000,1.000000,0.000000,1.000000,0.154167,0.005556,34.490278
25%,120446.000000,6.000000,133.000000,25.000000,15.221528,14.512500,58.429167
50%,240504.000000,12.000000,280.000000,25.000000,29.272917,31.010417,60.750694
75%,364395.000000,21.000000,421.000000,25.000000,45.255556,45.722917,63.334722
max,503577.000000,27.000000,533.000000,25.000000,78.269444,74.098611,138.350694


In [62]:
#Requirement: All Numerical columns basic statistics.
df_test.describe()

,IDX,REMAIN_LAYER_SEQ,REMAIN_OP_SEQ,WIP_QTY,PROCESSED_DAYS,REMAIN_DAYS,CYCLE_TIME
count,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000
mean,536728.912903,7.238710,165.248387,24.703226,43.022079,17.235423,60.257502
std,899.380471,4.748134,116.212058,2.230051,12.151342,11.714860,4.419071
min,535018.000000,1.000000,2.000000,1.000000,8.211806,0.078472,39.134722
25%,535974.500000,3.000000,57.000000,25.000000,32.217535,7.252604,58.651563
50%,536670.000000,7.000000,160.000000,25.000000,43.732292,15.133681,60.440278
75%,537520.750000,11.000000,272.000000,25.000000,53.270660,28.712847,62.535937
max,539293.000000,24.000000,476.000000,25.000000,66.272917,36.232639,72.343056


sweetviz

In [63]:
import sweetviz as sv
pairwise_analysis='off' #相關性和其他型別的資料關聯可能需要花費較長時間。如果超過了某個閾值，就需要設定這個引數為on或者off，以判斷是否需要分析資料相關性。

In [64]:
report_train = sv.analyze([df_train, 'train']) # 'train'是指會給這個資料集命名為train
report_train.show_html(filepath='Basic_train_report.html') # 儲存為html的格式

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)
Report Basic_train_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [65]:
#DATA_DATE', 'IDX', 'LOT_ID', 'STATUS', 'CHIPNAME', 'LAYER',
# 'REMAIN_LAYER_SEQ', 'OP_NO', 'REMAIN_OP_SEQ', 'PRIORITY', 'LOT_TYPE',
# 'WIP_QTY', 'WS_DATE', 'IS_MAIN_ROUTE', 'ACTUAL_WP_OUT',
# 'PROCESSED_DAYS', 'REMAIN_DAYS'
feature_config = sv.FeatureConfig(skip=['DATA_DATE','IDX','WS_DATE'],  # 要忽略哪個特徵
                                  force_cat=[  'STATUS', 'CHIPNAME', 'LAYER','OP_NO','PRIORITY', 'LOT_TYPE',  'WS_DATE','IS_MAIN_ROUTE','ACTUAL_WP_OUT'], # Categorical特徵
                                  force_num=['WIP_QTY','REMAIN_LAYER_SEQ','REMAIN_DAYS','PROCESSED_DAYS'], # Numerical特徵
                                  force_text='LOT_ID') # Text特徵
report_train_with_target = sv.analyze([df_train, 'train'],
                                     target_feat='CYCLE_TIME', # 加入特徵變數
                                     feat_cfg=feature_config) 
report_train_with_target.show_html(filepath='Basic_train_report_with_target.html')                                                                      

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)Report Basic_train_report_with_target.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.



In [66]:

compare_report = sv.compare([df_train, 'Training Data'], # 使用compare
                            [df_test, 'Test Data'],
                            'CYCLE_TIME',
                            feat_cfg=feature_config)

compare_report.show_html(filepath='Compare_train_test_report.html')    
#
# compare_report.show_notebook(  w=None, 
#                 h=None, 
#                 scale=None,
#                 layout='widescreen',
#                 filepath=None)

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:02 -> (00:00 left)Report Compare_train_test_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.



In [73]:
compare_subsets_report = sv.compare_intra(df_train,
                                          df_train['CYCLE_TIME']=='WAITING', # 給條件區分
                                          ['WAITING', 'PROCESSING'], # 為兩個子資料集命名 
                                          target_feat='CYCLE_TIME',
                                          feat_cfg=feature_config)

compare_subsets_report.show_html(filepath='Compare_male_female_report.html')




                                             |          | [  0%]   00:00 -> (? left)


[Summarizing dataframe]                      |          | [  0%]   00:00 -> (? left)

ZeroDivisionError: float division by zero

In [68]:
# A=data.query("category in ('AAR', 'AAU')")
# B=data.query("category not in ('AAR', 'AAU')")
# sv.compare([A, 'A'], [B, 'B']).show_html()